In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from glob import glob
from datetime import datetime
import sys
sys.path.append('/Users/w76m851/github_workspace/spicy-snow/')

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow


In [2]:
# %%time
files = sorted(glob('/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/*.nc'))

# Create parameter space
A = np.arange(1, 3.1, 0.5)
B = np.arange(0, 1.01, 0.1)
C = np.arange(0, 1.001, 0.01)

for f in files[6:]:
    ds_name = f.split('stacks/')[-1].split('.')[0]
    if 'Mores_2020' in ds_name:
        continue
    print(datetime.now(), f' -- starting {ds_name}')
    # Open dataset 
    ds_ = xr.open_dataset(f).load()
    dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
    td = abs(pd.to_datetime(dataset.time) - pd.to_datetime(dataset.attrs['lidar-flight-time']))
    closest_ts_idx = np.where(td == td.min())[0][0]
    closest_ts = dataset.time[closest_ts_idx]
    # Initialize RMSE arrays
    rmse_wet_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    rmse_no_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    
    # Brute-force loop
    for a in A:
        ds = calc_delta_cross_ratio(dataset, A=a, inplace=False)
        for b in B:
            ds = calc_delta_gamma(ds, B=b, inplace=False)
            for c in C:
                # print(f'A={a}; B={b}; C={c}')
                ds = clip_delta_gamma_outlier(ds)
                ds = calc_snow_index(ds)
                ds = calc_snow_index_to_snow_depth(ds, C=c, inplace=False)
                ds = id_newly_wet_snow(ds)
                ds = id_wet_negative_si(ds)
                ds = id_newly_frozen_snow(ds)
                ds = flag_wet_snow(ds)
                # Compare snow depths - mask wet snow
                mask_wet = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)) | 
                        ds['wet_snow'].sel(time=closest_ts).astype(bool))
                diff_wet = ds['lidar-sd'].where(mask_wet) - ds['snow_depth'].sel(time=closest_ts).where(mask_wet)
                rmse_wet = float(np.sqrt((diff_wet**2).sum()/len(diff_wet.values.flatten())))
                rmse_wet_flag.loc[a, b, c] = rmse_wet
                # Compare snow depths - no wet snow mask
                mask = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)))
                diff = ds['lidar-sd'].where(mask) - ds['snow_depth'].sel(time=closest_ts).where(mask)
                rmse = float(np.sqrt((diff**2).sum()/len(diff.values.flatten())))
                rmse_no_flag.loc[a,b,c] = rmse

    # After loop, save RMSE results per file
    rmse_wet_flag.to_netcdf(f'rmse_out/{ds_name}_wet_flag.nc')
    rmse_no_flag.to_netcdf(f'rmse_out/{ds_name}_no_flag.nc')
    

2023-03-05 19:54:30.735506  -- starting Little_Cottonwood_2021-03-18
2023-03-06 05:05:46.300686  -- starting Mores_2021-03-15


In [4]:
rmse_wet_flag.min()



<xarray.DataArray ()>
array(0.15777767)

In [ ]:
for t in ds.time:
    print(ds['s1'].sel(band='VH', time=t).values.sum())

In [21]:
test = xr.open_dataarray('rmse_out/Banner_2020-02')
test

<xarray.DataArray (A: 3, B: 6, C: 21)>
[378 values with dtype=float64]
Coordinates:
  * A        (A) float64 1.0 2.0 3.0
  * B        (B) float64 0.0 0.2 0.4 0.6 0.8 1.0
  * C        (C) float64 0.0 0.05 0.1 0.15 0.2 0.25 ... 0.8 0.85 0.9 0.95 1.0